### SetUp

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os
import string

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
keras = tf.keras

2024-03-15 18:17:39.481346: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 18:17:42.883054: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 18:17:42.893870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 18:17:57.004554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Load Dataset

In [3]:
dataset, info = tfds.load("glue/sst2", with_info = True)
print(info.features)
print(info.features['label'].num_classes)
print(info.features['label'].names)

FeaturesDict({
    'idx': int32,
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    'sentence': Text(shape=(), dtype=string),
})
2
['negative', 'positive']


#### Get the train and test Dataset

In [5]:
dataset_train, dataset_test = dataset['train'], dataset['test']
dataset_train

<_PrefetchDataset element_spec={'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sentence': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [19]:
# print some entries
for example in dataset_train.take(2):
    review, label = example['sentence'], example['label']
    print("Review: ",str(review.numpy()))
    print("Label: ",label.numpy())

Review:  b'for the uninitiated plays better on video with the sound '
Label:  0
Review:  b'like a giant commercial for universal studios , where much of the action takes place '
Label:  0


2024-03-15 18:35:28.861909: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [23]:
print(f"Train length: {len(dataset_train)}")
print(f"Test length: {len(dataset['test'])}")

Train length: 67349
Test length: 1821


In [33]:
train_reviews = []
train_labels = []

test_reviews = []
test_labels = []

for item in dataset_train.take(10000):
    train_reviews.append(str(item['sentence'].numpy()))
    train_labels.append(item['label'].numpy())
    
print(f"No. of train examples: {len(train_labels)}")

# print some examples
for i in range(33, 36):
    print("Review: ",train_reviews[i])
    print("Label: ",train_labels[i])

print("\n\n")
for item in dataset_test.take(1000):
    review, label = item['sentence'], item['label']
    test_reviews.append(str(review.numpy()))
    test_labels.append(label.numpy())

print(f"No of test example: {len(test_reviews)}")
for i in range(33, 36):
    print("Review: ",test_reviews[i])
    print("Label: ",test_labels[i])

2024-03-15 18:51:34.505694: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


No. of train examples: 10000
Review:  b"is as blasphemous and nonsensical as a luis bu\xc3\xb1uel film without the latter 's attendant intelligence , poetry , passion , and genius . "
Label:  0
Review:  b'at heart is a sweet little girl '
Label:  1
Review:  b"'s suspenseful enough for older kids but not too scary "
Label:  1



No of test example: 1000
Review:  b"it will come as no surprise that the movie is n't scary ."
Label:  -1
Review:  b'the production values are up there .'
Label:  -1
Review:  b'little more than a frothy vanity project .'
Label:  -1


### Tokenize the words and sequence the sentence

In [37]:
vocab_size = 4000
embedding_dim = 16
max_length = 50
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_reviews)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, "'": 2, 'the': 3, 'and': 4, 'a': 5, 'of': 6, 'b': 7, 'to': 8, "'s": 9, 'in': 10, 'that': 11, 'is': 12, 'it': 13, 'as': 14, "b'a": 15, 'with': 16, 'film': 17, 'movie': 18, 'for': 19, "b'the": 20, 'its': 21, 'an': 22, 'you': 23, 'this': 24, 'but': 25, 'on': 26, 'be': 27, "n't": 28, "b'is": 29, 'by': 30, "b'": 31, 'at': 32, 'not': 33, 'or': 34, 'about': 35, 'more': 36, 'one': 37, 'so': 38, 'than': 39, 'all': 40, 'out': 41, 'from': 42, 'his': 43, 'like': 44, 'have': 45, 'are': 46, 'good': 47, 'story': 48, 'up': 49, 'who': 50, 'most': 51, 'has': 52, 'too': 53, 'into': 54, 'your': 55, 'time': 56, 'characters': 57, 'just': 58, 'comedy': 59, 'their': 60, 'funny': 61, 'life': 62, 'i': 63, 'even': 64, 'well': 65, "b'of": 66, 'little': 67, 'much': 68, 'does': 69, 'if': 70, 'will': 71, 'very': 72, 'what': 73, "b'an": 74, 'can': 75, 'which': 76, 'been': 77, 'no': 78, 'love': 79, "b'that": 80, "''": 81, 'bad': 82, "b'to": 83, 'best': 84, 'work': 85, 'action': 86, 'humor': 87, 'way': 88,

### Pad the sequences

In [47]:
train_seq = tokenizer.texts_to_sequences(train_reviews)
train_padded = pad_sequences(train_seq, padding = padding_type,
                            truncating = trunc_type, maxlen = max_length)

test_seq = tokenizer.texts_to_sequences(test_reviews)
test_padded = pad_sequences(test_seq, padding = padding_type,
                           maxlen = max_length, truncating = trunc_type)

train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)
train_labels_final

array([0, 0, 1, ..., 0, 1, 1])